In [1]:
import pandas as pd
import numpy as np
import os, sys
from collections import defaultdict, Counter

In [4]:
df_session = pd.read_csv("data/session/session_reduced_2016-04-01.csv", sep = ";")
df_purchase = pd.read_csv("data/purchase/purchase_2016-04-01.csv", sep = ";")

df_session.shape

df_purchase.shape

session_dict = {}
for my_id,search_id, prop60 in df_session.values:
    key = (my_id, search_id)
    session_dict[key] = prop60.split(":")

tot = []
for my_id, search_id, product_id_buy in df_purchase.values:
    key = (my_id, search_id)
    if key in session_dict:
        top6 = session_dict[key]
        if product_id_buy in top6:
            pos = top6.index(product_id_buy)
            tot.append(pos)
        else:
            tot.append(100)

cnt = Counter(tot)

cnt.most_common()

#### find ranking change

ranking_dict = defaultdict(lambda: defaultdict(int))

session_dict = {}
for my_id,search_id, prop60 in df_session.values:
    key = (my_id, search_id)
    session_dict[key] = prop60.split(":")
    
    
for my_id, search_id, product_id_buy in df_purchase.values:
    key = (my_id, search_id)
    if key in session_dict:
        top6 = session_dict[key]
        if product_id_buy in top6:
            pos = top6.index(product_id_buy)
            ranking_dict[(search_id, product_id_buy)][pos]+=1
        else:
            ranking_dict[(search_id, product_id_buy)][100]+=1

out = []
for key, mydict in ranking_dict.items():
    if len(mydict) > 1:
        for k, v in mydict.items():
            out.append((key[0], key[1], k, v))
out_df = pd.DataFrame(out, columns = ['search_id', 'product_id', 'position', 'count'])
out_df.sort_values(['search_id', 'product_id', 'count'])
out_df.to_csv("data/position_stat.csv", index = False, header = True, sep = ";")